In [6]:
import time
import importlib
import numpy as np
import tensorflow as tf
from tensorflow.contrib.slim import fully_connected as fc
import matplotlib.pyplot as plt 
import glob
from collections import OrderedDict
%matplotlib inline
from datetime import datetime
import os
from pathlib import Path
import layer_def as ld
import BasicConvLSTMCell
#from BasicConvLSTMCell import BasicConvLSTMCell
from avg_g_model import *
from avg_d_model import *
importlib.reload(ld)

train_files = "/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/train"
test_files = "/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/test"
val_files = "/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/train"
#hyparameters

def set_save_name(name):
    """
    Edits all constants dependent on SAVE_NAME.

    @param name: The new save name.
    """
    #global SAVE_NAME, MODEL_SAVE_DIR, SUMMARY_SAVE_DIR, IMG_SAVE_DIR

    SAVE_NAME = name
    MODEL_SAVE_DIR = get_dir(os.path.join(SAVE_DIR, 'Models/', SAVE_NAME))
    SUMMARY_SAVE_DIR = get_dir(os.path.join(SAVE_DIR, 'Summaries/', SAVE_NAME))
    IMG_SAVE_DIR = get_dir(os.path.join(SAVE_DIR, 'Images/', SAVE_NAME))
    return SAVE_NAME, MODEL_SAVE_DIR,SUMMARY_SAVE_DIR,IMG_SAVE_DIR

def get_dir(directory):
    """
    Creates the given directory if it does not exist.

    @param directory: The path to the directory.
    @return: The path to the directory.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

/Users/gongbing/anaconda3/envs/GAN_practice/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/gongbing/anaconda3/envs/GAN_practice/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/gongbing/anaconda3/envs/GAN_practice/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/gongbing/ana

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('GeneratorModel: saved summaries')? (avg_g_model.py, line 292)

In [ ]:
class AVG(object):
    
        def __init__(self, learning_rate_g=0.00004, learning_rate_d=0.02,batch_size=32,
                     checkpoint_dir=None,context_frames=10,
                     sequence_length=20,adversarial=None,tranpose=[0,2,3,1],
                     train_height=32,train_width=32,padding_g="SAME",num_epochs=50,
                     stats_freq=10,summary_freq=100,img_save_freq=1000,
                     test_freq=5000,model_save_freq=10000):
            """
            @params tranpose:None or list<int>, if tranpose the raw dataset, which tensor shape is [sequence_length, height,width,channel]
            @params train_width/train_height: int, trian_height, train_width: the height and width of the patches to train on 
            @params padding_g:padding for convolutions in the generator model
            @params num_epochs: int, the number of epochs for iteration
            @params stats_freq: int, how often to print loss/train error stats, in # steps
            @params summary_sesq:int, how often to save the summaries, in # steps
            @params img_save_freq:int, how often to save generated images, in # steps
            @params test_freq:int, how often to test the model on test data, in # steps
            @params model_save_freq: how often to save the model, in # steps
            """
            self.learning_rate_g = learning_rate_g
            self.learning_rate_d = learning_rate_d
            self.batch_size = batch_size
            self.sequence_length = sequence_length
            self.context_frames = context_frames
            self.adversarial = adversarial
            self.transpose = transpose
            self.train_height = train_height 
            self.train_width = train_width
            self.padding_g = padding_g
            self.num_epochs = num_epochs
            self.stats_freq = stats_freq
            self.test_freq = test_freq
            self.summary_freq = summary_freq
            self.img_save_freq = img_save_freq
            self.model_save_freq = model_save_freq
            self.save_name,self.model_save_name,self.summary_save_dir,self.image_save_dir = set_save_name(cls.__name__)
                      
            
        def runner(self):
            self.load_checkpoints()
            self.build_graph()
            self.start_session_and_initialize()
            self.train()
            #self.test()
            self.save()
            
        
        def make_dataset(self,type="train"):
            """
            @param tranpose:None or list<int>, if tranpose the raw dataset, which tensor shape is [sequence_length, height,width,channel]
            @return: A interator
            """
            if type=="train": filenames = glob.glob("/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/train/*.tfrecords")
            if type=="val":filenames = glob.glob("/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/val/*.tfrecords")
            if type=="test":filenames = glob.glob("/Users/gongbing/PycharmProjects/video_prediction_savp/data/era5_size_64_64_3_3t_norm/test/*.tfrecords")
            def parser(serialized_example):
                    seqs = OrderedDict()
                    keys_to_features = {
                        # 'width': tf.FixedLenFeature([], tf.int64),
                        # 'height': tf.FixedLenFeature([], tf.int64),
                        'sequence_length': tf.FixedLenFeature([], tf.int64),
                        # 'channels': tf.FixedLenFeature([],tf.int64),
                        # 'images/encoded':  tf.FixedLenFeature([], tf.string)
                        'images/encoded': tf.VarLenFeature(tf.float32)
                    }
                    parsed_features = tf.parse_single_example(serialized_example, keys_to_features)
                    seq = tf.sparse_tensor_to_dense(parsed_features["images/encoded"])
                    images = tf.reshape(seq, [self.sequence_length,64, 64,3], name="reshape_new")
                    if self.tranpose:
                        images = tf.transpose(images,self.transpose)
                    seqs["images"] = images
                    return seqs
            dataset = tf.data.TFRecordDataset(filenames, buffer_size = 8 * 1024 * 1024)
            dataset = dataset.repeat(num_epochs)
            dataset = dataset.apply(tf.contrib.data.map_and_batch(
                    parser, self.batch_size, drop_remainder = True, num_parallel_calls = None))
            dataset = dataset.prefetch(self.batch_size) 
            iterator = dataset.make_initializable_iterator()
            return iterator

            
        def data_construct(self):
                  
            self.train_iterator = make_dataset(type="train",tranpose=self.tranpose,batch_size=self.batch_size)
            self.val_iterator = make_dataset(type="val",,tranpose=self.tranpose,batch_size=self.batch_size)
            self.test_iterator = make_dataset(type="test",tranpose=self.tranpose,batch_size=self.batch_size) 
            
            dat_shape = self.train_iterator.get_shape().tolist()
            self.height_test = dat_shape[1]
            self.width_test = dat_shape[2]
            self.channel = dat_shape[4]
                               
            with tf.name_scope('data'):
                self.input_frames_train = tf.placeholder(
                    tf.float32, shape=[None, self.height_train, self.width_train, self.channel * self.context_frames])
                self.gt_frames_train = tf.placeholder(
                    tf.float32, shape=[None, self.height_train, self.width_train, self.channel])

                self.input_frames_test = tf.placeholder(
                    tf.float32, shape=[None, self.height_test, self.width_test, self.channel * self.context_frames])
                self.gt_frames_test = tf.placeholder(
                    tf.float32, shape=[None, self.height_test, self.width_test, self.channel])

                # use variable batch_size for more flexibility
                self.batch_size_train = tf.shape(self.input_frames_train)[0]
                self.batch_size_test = tf.shape(self.input_frames_test)[0]

            ##
            # Scale network setup and calculation
            ##

            self.summaries_train = []
            self.scale_preds_train = []  # the generated images at each scale
            self.scale_gts_train = []  # the ground truth images at each scale
            self.d_scale_preds = []  # the predictions from the discriminator model

            self.summaries_test = []
            self.scale_preds_test = []  # the generated images at each scale
            self.scale_gts_test = []  # the ground truth images at each scale

            for scale_num in xrange(self.num_scale_nets):
                with tf.name_scope('scale_' + str(scale_num)):
                    with tf.name_scope('setup'):
                        ws = []
                        bs = []

                        # create weights for kernels
                        for i in xrange(len(self.scale_kernel_sizes[scale_num])):
                            ws.append(w([self.scale_kernel_sizes[scale_num][i],
                                         self.scale_kernel_sizes[scale_num][i],
                                         self.scale_layer_fms[scale_num][i],
                                         self.scale_layer_fms[scale_num][i + 1]]))
                            bs.append(b([self.scale_layer_fms[scale_num][i + 1]]))

                    with tf.name_scope('calculation'):
                        def calculate(height, width, inputs, gts, last_gen_frames):
                            # scale inputs and gts
                            scale_factor = 1. / 2 ** ((self.num_scale_nets - 1) - scale_num)
                            scale_height = int(height * scale_factor)
                            scale_width = int(width * scale_factor)

                            inputs = tf.image.resize_images(inputs, [scale_height, scale_width])
                            scale_gts = tf.image.resize_images(gts, [scale_height, scale_width])

                            # for all scales but the first, add the frame generated by the last
                            # scale to the input
                            if scale_num > 0:
                                last_gen_frames = tf.image.resize_images(
                                    last_gen_frames,[scale_height, scale_width])
                                inputs = tf.concat(3, [inputs, last_gen_frames])

                            # generated frame predictions
                            preds = inputs

                            # perform convolutions
                            with tf.name_scope('convolutions'):
                                for i in xrange(len(self.scale_kernel_sizes[scale_num])):
                                    # Convolve layer
                                    preds = tf.nn.conv2d(
                                        preds, ws[i], [1, 1, 1, 1], padding=self.padding_g)

                                    # Activate with ReLU (or Tanh for last layer)
                                    if i == len(self.scale_kernel_sizes[scale_num]) - 1:
                                        preds = tf.nn.tanh(preds + bs[i])
                                    else:
                                        preds = tf.nn.relu(preds + bs[i])

                            return preds, scale_gts

                        ##
                        # Perform train calculation
                        ##

                        # for all scales but the first, add the frame generated by the last
                        # scale to the input
                        if scale_num > 0:
                            last_scale_pred_train = self.scale_preds_train[scale_num - 1]
                        else:
                            last_scale_pred_train = None

                        # calculate
                        train_preds, train_gts = calculate(self.height_train,
                                                           self.width_train,
                                                           self.input_frames_train,
                                                           self.gt_frames_train,
                                                           last_scale_pred_train)
                        self.scale_preds_train.append(train_preds)
                        self.scale_gts_train.append(train_gts)

                        # We need to run the network first to get generated frames, run the
                        # discriminator on those frames to get d_scale_preds, then run this
                        # again for the loss optimization.
                        if self.adversarial:
                            self.d_scale_preds.append(tf.placeholder(tf.float32, [None, 1]))

                        ##
                        # Perform test calculation
                        ##

                        # for all scales but the first, add the frame generated by the last
                        # scale to the input
                        if scale_num > 0:
                            last_scale_pred_test = self.scale_preds_test[scale_num - 1]
                        else:
                            last_scale_pred_test = None

                        # calculate
                        test_preds, test_gts = calculate(self.height_test,
                                                         self.width_test,
                                                         self.input_frames_test,
                                                         self.gt_frames_test,
                                                         last_scale_pred_test)
                        self.scale_preds_test.append(test_preds)
                        self.scale_gts_test.append(test_gts)
                        
                        
        def generator_training_construct(self):
            with tf.name_scope('train'):
                # global loss is the combined loss from every scale network
                self.global_loss = combined_loss(self.scale_preds_train,
                                                 self.scale_gts_train,
                                                 self.d_scale_preds)
                self.global_step = tf.Variable(0, trainable=False)
                self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate_g, name='optimizer')
                self.train_op = self.optimizer.minimize(self.global_loss,
                                                        global_step=self.global_step,
                                                        name='train_op')


        def loss_summary_writer_construct(self):
            # train loss summary
            loss_summary = tf.scalar_summary('train_loss_G', self.global_loss)
            self.summaries_train.append(loss_summary)
 

        def performance_eval_construct(self):
            with tf.name_scope('error'):
                # error computation
                # get error at largest scale
                self.psnr_error_train = psnr_error(self.scale_preds_train[-1],
                                                   self.gt_frames_train)
                self.sharpdiff_error_train = sharp_diff_error(self.scale_preds_train[-1],
                                                              self.gt_frames_train)
                self.psnr_error_test = psnr_error(self.scale_preds_test[-1],
                                                  self.gt_frames_test)
                self.sharpdiff_error_test = sharp_diff_error(self.scale_preds_test[-1],
                                                             self.gt_frames_test)
                # train error summaries
                summary_psnr_train = tf.scalar_summary('train_PSNR',
                                                       self.psnr_error_train)
                summary_sharpdiff_train = tf.scalar_summary('train_SharpDiff',
                                                            self.sharpdiff_error_train)
                self.summaries_train += [summary_psnr_train, summary_sharpdiff_train]

                # test error
                summary_psnr_test = tf.scalar_summary('test_PSNR',
                                                      self.psnr_error_test)
                summary_sharpdiff_test = tf.scalar_summary('test_SharpDiff',
                                                           self.sharpdiff_error_test)
                self.summaries_test += [summary_psnr_test, summary_sharpdiff_test]

            # add summaries to visualize in TensorBoard
            self.summaries_train = tf.merge_summary(self.summaries_train)
            self.summaries_test = tf.merge_summary(self.summaries_test)
        
        
        def train_generator(self,batch,discriminator=None):
            """
            Runs a training step using the global loss on each of the scale networks.

            @param batch: An array of shape
                          [self.batch_size x self.height x self.width x (3 * (self.context_frames + 1))].
                          The input and output frames, concatenated along the channel axis (index 3).
            @param discriminator: The discriminator model. Default = None, if not adversarial.

            @return: The global step.
            """
            ##
            # Split into inputs and outputs
            ##

            input_frames = batch[:, :, :, :-3]
            gt_frames = batch[:, :, :, -3:]

            ##
            # Train
            ##

            feed_dict = {self.input_frames_train: input_frames, self.gt_frames_train: gt_frames}

            if self.adversarial:
                # Run the generator first to get generated frames
                scale_preds = self.sess.run(self.scale_preds_train, feed_dict=feed_dict)

                # Run the discriminator nets on those frames to get predictions
                d_feed_dict = {}
                for scale_num, gen_frames in enumerate(scale_preds):
                    d_feed_dict[discriminator.scale_nets[scale_num].input_frames] = gen_frames
                d_scale_preds = self.sess.run(discriminator.scale_preds, feed_dict=d_feed_dict)

                # Add discriminator predictions to the
                for i, preds in enumerate(d_scale_preds):
                    feed_dict[self.d_scale_preds[i]] = preds

            _, global_loss, global_psnr_error, global_sharpdiff_error, global_step, summaries = \
                self.sess.run([self.train_op,
                               self.global_loss,
                               self.psnr_error_train,
                               self.sharpdiff_error_train,
                               self.global_step,
                               self.summaries_train],
                              feed_dict=feed_dict)

            ##
            # User output
            ##
            if global_step % self.stats_freq == 0:
                print('GeneratorModel : Step ', global_step)
                print('                 Global Loss    : ', global_loss)
                print('                 PSNR Error     : ', global_psnr_error)
                print('                 Sharpdiff Error: ', global_sharpdiff_error)
            if global_step % self.summary_freq == 0:
                self.summary_writer.add_summary(summaries, global_step)
                print ('GeneratorModel: saved summaries')
            if global_step % self.img_save_freq == 0:
                print ('-' * 30)
                print ('Saving images...')

                # if not adversarial, we didn't get the preds for each scale net before for the
                # discriminator prediction, so do it now
                if not self.adversarial:
                    scale_preds = self.sess.run(self.scale_preds_train, feed_dict=feed_dict)

                # re-generate scale gt_frames to avoid having to run through TensorFlow.
                scale_gts = []
                for scale_num in xrange(self.num_scale_nets):
                    scale_factor = 1. / 2 ** ((self.num_scale_nets - 1) - scale_num)
                    scale_height = int(self.height_train * scale_factor)
                    scale_width = int(self.width_train * scale_factor)

                    # resize gt_output_frames for scale and append to scale_gts_train
                    scaled_gt_frames = np.empty([self.batch_size, scale_height, scale_width, 3])
                    for i, img in enumerate(gt_frames):
                        # for skimage.transform.resize, images need to be in range [0, 1], so normalize
                        # to [0, 1] before resize and back to [-1, 1] after
                        sknorm_img = (img / 2) + 0.5
                        resized_frame = resize(sknorm_img, [scale_height, scale_width, 3])
                        scaled_gt_frames[i] = (resized_frame - 0.5) * 2
                    scale_gts.append(scaled_gt_frames)

                # for every clip in the batch, save the inputs, scale preds and scale gts
                for pred_num in xrange(len(input_frames)):
                    pred_dir = get_dir(os.path.join(self.img_save_dir, 'Step_' + str(global_step),
                                                      str(pred_num)))

                    # save input images
                    for frame_num in xrange(self.context_frames):
                        img = input_frames[pred_num, :, :, (frame_num * 3):((frame_num + 1) * 3)]
                        imsave(os.path.join(pred_dir, 'input_' + str(frame_num) + '.png'), img)

                    # save preds and gts at each scale
                    # noinspection PyUnboundLocalVariable
                    for scale_num, scale_pred in enumerate(scale_preds):
                        gen_img = scale_pred[pred_num]

                        path = os.path.join(pred_dir, 'scale' + str(scale_num))
                        gt_img = scale_gts[scale_num][pred_num]

                        imsave(path + '_gen.png', gen_img)
                        imsave(path + '_gt.png', gt_img)

                print ('Saved images!')
                print ('-' * 30)
            return global_step
        
        
        def train_discriminator(self):
            pass
    
        def train(self,batch):
            """
            Runs a training loop on the model networks.
            """
            for i in xrange(self.num_epochs):
                if self.adversarial:
                    # update discriminator
                    batch = get_train_batch()
                    print ('Training discriminator...')
                    self.d_model.train_step(batch, self.g_model)

                # update generator
                batch = get_train_batch() #Bing replace with iterator
                print ('Training generator...')
                self.global_step = self.g_model.train_step(
                    batch, discriminator=(self.d_model if self.adversarial else None))

                # save the models
                if self.global_step % c.model_save_freq == 0:
                    print ('-' * 30)
                    print ('Saving models...')
                    self.saver.save(self.sess,
                                    self.model_save_dir + 'model.ckpt',
                                    global_step=self.global_step)
                    print ('Saved models!')
                    print ('-' * 30)

                # test generator model
                if self.global_step % c.test_freq == 0:
                    self.test()
                    
                    
        def test(self):
            """
            Runs one test step on the generator network.
            """
            batch = get_test_batch(self.batch_size, num_rec_out=self.num_test_rec)
            self.g_model.test_batch(
                batch, self.global_step, num_rec_out=self.num_test_rec)
            
                  
        def load_checkpoints(self):
            pass
        
        
    
        

        
        def start_session_and_initialize(self):
            print 'Init variables...'
            self.saver = tf.train.Saver(keep_checkpoint_every_n_hours=2)
            self.sess.run(tf.global_variables_initializer())
            return None

                    
        def build_graph(self):
            tf.reset_default_graph()
            tf.set_random_seed(12345)
            self.data_construct()
            self.global_step = tf.train.get_or_create_global_step()
            self.generator_training_construct()
            self.loss_summary_writer_construct()
            self.performance_eval_construct()
         
          
        def predict(self):
            pass
               

        
        
            
            
    